<a href="https://colab.research.google.com/github/xoyeon/hongong-mldl/blob/main/05_3_%ED%8A%B8%EB%A6%AC%EC%9D%98_%EC%95%99%EC%83%81%EB%B8%94.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 정형 데이터와 비정형 데이터
* 정형 데이터(structured data)
  - 가지런히 어떤 구조로 정리되어 있는 데이터
  - CSV나 데이터베이스, 혹은 엑셀에 저장하기 쉬움
> 앙상블 학습(ensemble learning)은 정형 데이터를 다루는 데 가장 뛰어난 성과를 냄

* 비정형 데이터(unstructured data)
  - 데이터베이스나 엑셀로 표현하기 어려운 것들
  - 텍스트 데이터, 디지털 사진, 디지털 음악 등
> 신경망 알고리즘

# 랜덤 포레스트(Random Forest)
: 각 트리를 훈련하기 위한 데이터를 랜덤하게 만듦

* 부트스트랩 샘플(boostrap sample)은 데이터 세트에서 중복을 허용하여 데이터를 샘플링하는 방식

> **RandomForestClassifier**는 기본적으로 전체 특성 개수의 제곱근만큼의 특성을 선택
- 각 트리의 클래스별 확률을 평균하여 가장 높은 확률을 가진 클래스를 예측

> **RandomForestRegressor**는 전체 특성을 사용하여 각 트리의 예측을 평균

In [9]:
# 데이터 불러오기
import pandas as pd
import numpy as np

wine = pd.read_csv('https://bit.ly/wine_csv_data')

In [10]:
# RandomForestClassifier 클래스를 화이트 와인을 분류하는 문제에 적용
from sklearn.model_selection import train_test_split
data = wine[['alcohol','sugar','pH']].to_numpy()
target = wine['class'].to_numpy()
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [12]:
# cross_validate() 함수를 사용해 교차 검증 수행
from sklearn.model_selection import cross_validate
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(rf, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9973541965122431 0.8905151032797809


In [13]:
# 앞의 랜덤 포레스트 모델을 훈련 세트에 훈련한 후 특성 중요도 출력
rf.fit(train_input, train_target)
print(rf.feature_importances_)

[0.23167441 0.50039841 0.26792718]


In [14]:
# 부트스트랩 샘플에 포함되지 않고 남는 샘플인 OOB(Out Of Bag)의 점수 출력
rf = RandomForestClassifier(oob_score=True, n_jobs=-1, random_state=42)
rf.fit(train_input, train_target)
print(rf.oob_score_)

0.8934000384837406


# 엑스트라 트리(Extra Trees)
- 기본적으로 100개의 결정 트리 훈련
- 전체 특성 중에 일부 특성을 랜덤하게 선택하여 노드를 분할하는 데 사용
- 단, 부트스트랩 샘플을 사용하지 않음
splitter='random'인 결정 트리

In [17]:
# ExtraTreesClassifeir의 교차 검증 점수 확인
from sklearn.ensemble import ExtraTreesClassifier
et = ExtraTreesClassifier(n_jobs=-1, random_state=42)
scores = cross_validate(et, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9974503966084433 0.8887848893166506


In [18]:
# 훈련 후 특성 중요도 출력
et.fit(train_input, train_target)
print(et.feature_importances_)

[0.20183568 0.52242907 0.27573525]


# 그래이디언트 부스팅(Gradient boosting)
깊이가 얕은 결정 트리를 사용하여 이전 트리의 오차를 보완하는 방식
- 기본적으로 깊이가 3인 결정 트리를 100개 사용
- 과대적합에 강하고 일반적으로 높은 일반화 성능을 기대할 수 있음
  > **분류**에서는 **로지스틱 손실 함수** 사용   
  **회귀**에서는 평균 제곱 오차 함수 사용

In [20]:
# GradientBoostingClassifier를 사용해 와인 데이터셋의 교차 검증 점수 확인
from sklearn.ensemble import GradientBoostingClassifier
gb = GradientBoostingClassifier(random_state=42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8881086892152563 0.8720430147331015


In [21]:
# 학습률 증가시키고 트리 개수 늘리기
gb = GradientBoostingClassifier(n_estimators=500, learning_rate=0.2, random_state=42)
scores = cross_validate(gb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9464595437171814 0.8780082549788999


In [22]:
# 모델 훈련 후 특성 중요도 출력
gb.fit(train_input, train_target)
print(gb.feature_importances_)

[0.15872278 0.68010884 0.16116839]


# 히스토그램 기반 그레이디언트 부스팅(Histogram-based Gradient Boosting)
> 정형 데이터를 다루는 머신러닝 알고리즘 중에 가장 인기가 높음

- 번저 입력 특성을 256개의 구간으로 나눔 > 노드를 분할할 때 최적의 분할을 매우 빠르게 찾을 수 있음
- 256개의 구간 중에서 하나를 떼어 놓고 누락된 값을 위해서 사용

In [26]:
# HistGradientBoostingClassifier 클래스 적용하기
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
hgb = HistGradientBoostingClassifier(random_state=42)
scores = cross_validate(hgb, train_input, train_target,
                        return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.9321723946453317 0.8801241948619236


In [27]:
# permutation_importance() 함수를 사용하여 히스토그램 기반 그레이디언트 부스팅의 특성 중요도 계산
from sklearn.inspection import permutation_importance
hgb.fit(train_input, train_target)
result = permutation_importance(hgb, train_input, train_target, 
                                n_repeats=10, random_state=42, n_jobs=-1)
print(result.importances_mean)

[0.08876275 0.23438522 0.08027708]


In [28]:
# HistGradientBoostingClassifier를 사용해 테스트 세트에서의 성능을 최종 확인
hgb.score(test_input, test_target)

0.8723076923076923

In [29]:
# XGBoost 사용하여 교차 검증 점수 확인하기
from xgboost import XGBClassifier
xgb = XGBClassifier(tree_method='hist', random_state=42)
scores = cross_validate(xgb, train_input, train_target,
                        return_train_score=True)
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

0.8824322471423747 0.8726214185237284


In [32]:
# LightGBM 사용하여 점수 확인
from lightgbm import LGBMClassifier
lgb = LGBMClassifier(random_state=42)
scores = cross_validate(lgb, train_input, train_target,
                        return_train_score=True, n_jobs=-1)
print(np.mean(scores['test_score']), np.mean(scores['test_score']))

0.8789710890649293 0.8789710890649293


# 정리
## 사이킷런
* 랜덤 포레스트 : 부트스트랩 샘플 사용. 대표 앙상블 학습 알고리즘
* 엑스트라 트리 : 결정 트리의 노드를 랜덤하게 분할
* 그레이디언트 부스팅 : 이진 트리의 손실을 보완하는 식으로 얕은 결정 트리를 연속하여 추가
* 히스토그램 기반 그레이디언트 부스팅 : 훈련 데이터를 256개 정수 구간으로 나누어 빠르고 높은 성능을 냄

## 그외 라이브러리
* XGBoost
* LightGBM